<a href="https://colab.research.google.com/github/Sanz-8/Jet-Classification-using-Machine-Learning/blob/main/Resnet.50ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U "datasets>=2.14.6"

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Activation,BatchNormalization,Dropout,GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from huggingface_hub import login
from google.colab import userdata

token=userdata.get('HF_TOKEN')
login(token=token)


In [ ]:
from datasets import load_dataset
ds = load_dataset("dl4phys/top_tagging_images")

In [6]:
#This function is to convert the image to tensor
def preprocess_pil(pil_image,target_size=(224, 224)):

    pil_image = pil_image.convert("RGB") #Converting image into three channels

    pil_image = pil_image.resize(target_size) #Making sure that the size of the image is compatible with ResNet50(224,224)

    img_array = np.array(pil_image)

    img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32) # Converting it into tensors

    return img_tensor

In [21]:
# Due to limited computational power, training all the data with ResNet is not feasible. So, let's train the model with 100k, the validation and test datasets is 20k
train_indices=np.random.choice(1200000,100000,replace=False)
val_indices=np.random.choice(400000,20000,replace=False)
test_indices=np.random.choice(400000,20000,replace=False)


In [8]:
# Here the function performs preprocessing required by Resnet50
def preprocess_generator(data_type,indices):
    for (img,lbl) in zip(ds[data_type][indices]['image'],ds[data_type][indices]['label']):
        x = preprocess_pil(img)
        yield preprocess_input(x),lbl


In [22]:
# The training data is streamline for efficient computation by batching the data.Also, the size of the input data is converted into (224,224,3)
train_ds = tf.data.Dataset.from_generator(
    lambda:preprocess_generator('train',train_indices),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
).cache().repeat().batch(256).prefetch(tf.data.AUTOTUNE)


In [23]:
# Similar process is done for validation datasets
val_ds = tf.data.Dataset.from_generator(
    lambda:preprocess_generator('validation',val_indices),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
).batch(256).prefetch(tf.data.AUTOTUNE)


In [24]:
# This is for test datsets
test_ds = tf.data.Dataset.from_generator(
    lambda:preprocess_generator('test',test_indices),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
).batch(256).prefetch(tf.data.AUTOTUNE)


In [12]:
# Defining the ResNet50 with weights from 'imagenet'
res_model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
#Freezing the layers upto 169 and allowing training the last five layers
for l in res_model.layers[:170]:
  l.trainable=False

In [14]:
for l in res_model.layers[160:]:
  print(l.trainable)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True


In [15]:
# Adding Dense Layer on top of the ResNet50 model
model=Sequential()
model.add(res_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))


In [16]:
# Using Adam Optimizer with learning rate=1e-5
model.compile(optimizer=Adam(learning_rate=1e-5),loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
#Reduce learning rate by factor of 0.5 when the validation accuracy decreases for three consecutive epochs
reduce_alpha= ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)
#Stop training when the validation accuracy decreases for three consecutive epochs
stop_early=EarlyStopping(monitor='val_accuracy',patience=3,restore_best_weights=True)

In [ ]:
#raining the model
hist=model.fit(train_ds, validation_data=val_ds,epochs=20,steps_per_epoch=100_000//256,callbacks=[reduce_alpha,stop_early])

In [ ]:
y_hat=model.predict(test_ds)

    187/Unknown 9477s 51s/step

In [ ]:
y_hat=(y_hat>=0.5).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(ds['test'][test_indices]['label'],y_hat))

In [ ]:
plt.plot(hist.history['accuracy'],'ro',linestyle='dashed')
plt.plot(hist.history['val_accuracy'],'bo',linestyle='dashed')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

Below are the training , validataion and test accuracy for different trainable layaers

# Datasets(100k)
alpha=1e-5,epochs =5,Trainable layer[165:],training_accuracy=0.8837, validation accuracy= 0.8893 and test_accuracy=0.8877  
alpha=1e-5,epochs=5,Trainable layer[150:],training_accuracy=0.9025, validation accuracy= 0.8937 and test_accuracy=0.892  
alpha=1e-5, epochs=9,Trainable layer[150:],training_accuracy=0.9254, validation accuracy= 0.8932 and test_accuracy=0.89576


# Datasets(200k)
alpha=1e-6,epochs =5,Trainable layer[165:],training_accuracy=0.8837, validation accuracy= 0.8893 and test_accuracy=0.8877  

In [ ]:
#Saving the model on the drive
from google.colab import drive
drive.mount('/content/drive')
model.save('/content/drive/MyDrive/my_models/model_res.h5')


In [19]:
from google.colab import drive
from tensorflow.keras.models import load_model
drive.mount('/content/drive')
model = load_model('/content/drive/MyDrive/my_models/model_res.h5')


Mounted at /content/drive
